# Connecting to the Database

Defines the connection parameters, connects to an .sqlite database, and creates an .sqlite database if it does not exit.

In [2]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [4]:
connection = create_connection("sm_app.sqlite")

Connection to SQLite DB successful


# Defining function to try query and return the output if it fails or succeeds

In [5]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

# Creating Tables

## Users Table

This table will autoincrement the id. As you will see, this id will be referenced as user_id in other tables.

In [25]:
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER,
    gender TEXT,
    nationality TEXT
);
"""

In [26]:
execute_query(connection, create_users_table)

Query executed successfully


## Posts Table

In [27]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS posts(
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  title TEXT NOT NULL, 
  description TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id)
);
"""

In [28]:
execute_query(connection, create_posts_table)

Query executed successfully


## Comments Table & Likes Table

In [29]:
create_comments_table = """
CREATE TABLE IF NOT EXISTS comments (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  text TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  post_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

In [30]:
create_likes_table = """
CREATE TABLE IF NOT EXISTS likes (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  user_id INTEGER NOT NULL, 
  post_id integer NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""


In [31]:
execute_query(connection, create_comments_table)  
execute_query(connection, create_likes_table) 

Query executed successfully
Query executed successfully


# Populating the Tables

## Create the Users

In [39]:
create_users = """
INSERT INTO
  users (name, age, gender, nationality)
VALUES
  ('James', 25, 'male', 'USA'),
  ('Leila', 32, 'female', 'France'),
  ('Brigitte', 35, 'female', 'England'),
  ('Mike', 40, 'male', 'Denmark'),
  ('Elizabeth', 21, 'female', 'Canada');
"""

In [40]:
execute_query(connection, create_users)

Query executed successfully


## Create their Posts

In [34]:
create_posts = """
INSERT INTO
  posts (title, description, user_id)
VALUES
  ("Happy", "I am feeling very happy today", 1),
  ("Hot Weather", "The weather is very hot today", 2),
  ("Help", "I need some help with my work", 2),
  ("Great News", "I am getting married", 1),
  ("Interesting Game", "It was a fantastic game of tennis", 5),
  ("Party", "Anyone up for a late-night party today?", 3);
"""

In [35]:
execute_query(connection, create_posts)

Query executed successfully


## Create their Comments and their Likes

In [36]:
create_comments = """
INSERT INTO
  comments (text, user_id, post_id)
VALUES
  ('Count me in', 1, 6),
  ('What sort of help?', 5, 3),
  ('Congrats buddy', 2, 4),
  ('I was rooting for Nadal though', 4, 5),
  ('Help with your thesis?', 2, 3),
  ('Many congratulations', 5, 4);
"""

In [37]:
create_likes = """
INSERT INTO
  likes (user_id, post_id)
VALUES
  (1, 6),
  (2, 3),
  (1, 5),
  (5, 4),
  (2, 4),
  (4, 2),
  (3, 6);
"""

In [38]:
execute_query(connection, create_comments)
execute_query(connection, create_likes)

Query executed successfully
Query executed successfully


# Selecting Records

Similar to the function `execute_query()`, `execute_read_query()` also takes the 'connection' parameter and passes `str` queries through the argument. Note that the built-in function `.fetchall()` returns a list of tuples where each typle is mapped to the corresponding row in the retrieved records.

In [43]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

Define the `SELECT` line and pass that as the query.

In [44]:
select_users = "SELECT * FROM users"

In [45]:
users = execute_read_query(connection, select_users)

Because the results are returned tuple form in a list, to arrive at a line-by-line output, the return must be looped through. Note that it returns rows.

In [48]:
for user in users:
    print(user)

(1, 'James', 25, 'male', 'USA')
(2, 'Leila', 32, 'female', 'France')
(3, 'Brigitte', 35, 'female', 'England')
(4, 'Mike', 40, 'male', 'Denmark')
(5, 'Elizabeth', 21, 'female', 'Canada')
(6, 'James', 25, 'male', 'USA')
(7, 'Leila', 32, 'female', 'France')
(8, 'Brigitte', 35, 'female', 'England')
(9, 'Mike', 40, 'male', 'Denmark')
(10, 'Elizabeth', 21, 'female', 'Canada')
(11, 'James', 25, 'male', 'USA')
(12, 'Leila', 32, 'female', 'France')
(13, 'Brigitte', 35, 'female', 'England')
(14, 'Mike', 40, 'male', 'Denmark')
(15, 'Elizabeth', 21, 'female', 'Canada')


# Updating Table Records

Changing the records requires an `UPDATE`, `SET`, and `WHERE` sequence.
- `UPDATE` defines the table
- `SET` defines the content of that table (indicates under which column)
- `WHERE` defines the row in which to insert the content

In [49]:
update_post_description = """
UPDATE
  posts
SET
  description = "The weather has become pleasant now"
WHERE
  id = 2
"""

In [50]:
execute_query(connection, update_post_description)

Query executed successfully


# Deleting Table Records

Unlike updates, deletes are simpler because no content needs to be passed. All this requires is a reference to the column name and the row identifier.

In [51]:
delete_comment = "DELETE FROM comments WHERE id = 5"

In [52]:
execute_query(connection, delete_comment)

Query executed successfully


Check to see if the update and delete have taken.

In [56]:
select_posts = "SELECT * FROM posts"

In [57]:
select_comments = "SELECT * FROM comments"

In [58]:
posts = execute_read_query(connection, select_posts)

In [60]:
comments = execute_read_query(connection, select_comments)

In [61]:
for post in posts:
    print(post)

(1, 'Happy', 'I am feeling very happy today', 1)
(2, 'Hot Weather', 'The weather has become pleasant now', 2)
(3, 'Help', 'I need some help with my work', 2)
(4, 'Great News', 'I am getting married', 1)
(5, 'Interesting Game', 'It was a fantastic game of tennis', 5)
(6, 'Party', 'Anyone up for a late-night party today?', 3)
(7, 'Happy', 'I am feeling very happy today', 1)
(8, 'Hot Weather', 'The weather is very hot today', 2)
(9, 'Help', 'I need some help with my work', 2)
(10, 'Great News', 'I am getting married', 1)
(11, 'Interesting Game', 'It was a fantastic game of tennis', 5)
(12, 'Party', 'Anyone up for a late-night party today?', 3)


In [63]:
posts[1]

(2, 'Hot Weather', 'The weather has become pleasant now', 2)

In [62]:
for comment in comments:
    print(comment)

(1, 'Count me in', 1, 6)
(2, 'What sort of help?', 5, 3)
(3, 'Congrats buddy', 2, 4)
(4, 'I was rooting for Nadal though', 4, 5)
(6, 'Many congratulations', 5, 4)
(7, 'Count me in', 1, 6)
(8, 'What sort of help?', 5, 3)
(9, 'Congrats buddy', 2, 4)
(10, 'I was rooting for Nadal though', 4, 5)
(11, 'Help with your thesis?', 2, 3)
(12, 'Many congratulations', 5, 4)


In [64]:
comments[4]

(6, 'Many congratulations', 5, 4)

Note that the id above is `6` and not `5`. We deleted `id = 5`.

# NOTE W02D1: Generic Python Libraries for SQL

In [69]:
import psycopg2

In [70]:
# Connect to your postgres DB
conn = psycopg2.connect("dbname=lsantos user=postgres")

In [71]:
# Open a cursor to perform database operations
cur = conn.cursor()

In [73]:
# Execute a query
cur.execute("SELECT * FROM )

# Retrieve query results
records = cur.fetchall()

SyntaxError: EOL while scanning string literal (<ipython-input-73-3c3d8bac2a48>, line 2)

In [68]:
DATABASES = {
    "default": {
        "ENGINE": "django.db.backends.postgresql_psycopg2",
        "NAME": "lsantos",
        "USER": "lsantos",
        "PASSWORD": "",
        "HOST": "Applications/Postgres.app/Contents/Versions/14/bin/psql",
        "PORT": "5432",
    }
}

DatabaseError: ODBC error
state: 01000
native error code: 0
message: [unixODBC][Driver Manager]Can't open lib 'PostgreSQL' : file not found